# This case study consists of 1000 marks and the solution file is expected to be submitted in the upcoming session for evaluation.

# Combining Datasets: Merge and Join

One essential feature offered by Pandas is its high-performance, in-memory join and merge operations.
If you have ever worked with databases, you should be familiar with this type of data interaction.
The main interface for this is the ``pd.merge`` function, and we'll see few examples of how this can work in practice.

For convenience, we will start by redefining the ``display()`` functionality from the previous section:

In [1]:
import pandas as pd
import numpy as np


## Specification of the Merge Key

We've already seen the default behavior of ``pd.merge()``: it looks for one or more matching column names between the two inputs, and uses this as the key.
However, often the column names will not match so nicely, and ``pd.merge()`` provides a variety of options for handling this.

## Example: US States Data

Merge and join operations come up most often when combining data from different sources.
Here we will consider an example of some data about US states and their populations.
The data files can be found at http://github.com/jakevdp/data-USstates/:

In [2]:
# Following are shell commands to download the data
! mkdir -p data
!cd data ; curl -O  https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv ; cd ..
!cd data ; curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv ; cd ..

The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.
The filename, directory name, or volume label syntax is incorrect.


In [3]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Let's take a look at the three datasets, using the Pandas ``read_csv()`` function:

In [5]:
import os
os.chdir('C:/Users/tmutengwa/Documents/data')

In [7]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [8]:
display(pop.head())
display(areas.head())
display(abbrevs.head())

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


Given this information, say we want to compute a relatively straightforward result: rank US states and territories by their 2010 population density.
We clearly have the data here to find this result, but we'll have to combine the datasets to find the result.

We'll start with a many-to-one merge that will give us the full state name within the population ``DataFrame``.
We want to merge based on the ``state/region``  column of ``pop``, and the ``abbreviation`` column of ``abbrevs``.
We'll use ``how='outer'`` to make sure no data is thrown away due to mismatched labels.

## Merge the 2 dataframes, pop and abbrevs such that all rows in both tables are included. Drop any duplicate column in the merged dataframe.

In [12]:
df1 = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
df1 = df1.drop('abbreviation', axis =1) # drop duplicate info
df1.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


Let's double-check whether there were any mismatches here, which we can do by looking for rows with nulls:

## Look for any rows with null values in the new dataframe

In [13]:
df1.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

## Print out the rows in the merged dataframe that has null values for population.

It appears that all the null population values are from Puerto Rico prior to the year 2000; this is likely due to this data not being available from the original source.

More importantly, we see also that some of the new ``state`` entries are also null, which means that there was no corresponding entry in the ``abbrevs`` key!
Let's figure out which regions lack this match:

In [14]:
df1.loc[df1['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

We can quickly infer the issue: our population data includes entries for Puerto Rico (PR) and the United States as a whole (USA), while these entries do not appear in the state abbreviation key.
We can fix these quickly by filling in appropriate entries:

In [16]:
df1.loc[df1['state/region'] == 'PR', 'state'] = 'Puerto Rico'
df1.loc[df1['state/region'] == 'USA', 'state'] = 'United States'
df1.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

No more nulls in the ``state`` column: we're all set!

Now we can merge the result with the area data using a similar procedure.
Examining our results, we will want to join on the ``state`` column in both:

In [32]:
df2 = pd.merge(df1, areas, on='state', how='left')
df2.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


## Repeat the process of checking for nulls in the area field for each state in the merged dataframe and deal with them appropriately.

In [33]:
df2.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [34]:
df2[df2['area (sq. mi)'].isnull()]

,state/region,ages,year,population,state,area (sq. mi)
2496,USA,under18,1990,64218512.0,United States,NaN
2497,USA,total,1990,249622814.0,United States,NaN
2498,USA,total,1991,252980942.0,United States,NaN
2499,USA,under18,1991,65313018.0,United States,NaN
2500,USA,under18,1992,66509177.0,United States,NaN
2501,USA,total,1992,256514231.0,United States,NaN
2502,USA,total,1993,259918595.0,United States,NaN
2503,USA,under18,1993,67594938.0,United States,NaN
2504,USA,under18,1994,68640936.0,United States,NaN
2505,USA,total,1994,263125826.0,United States,NaN


In [35]:
df2['state'][df2['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [36]:
df2.dropna(inplace=True)
df2.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


## Select the portion of the data corresponding with the year 2000, and the total population.

In [37]:
FY00 = df2.query("year == 2000 & ages == 'total'")
FY00.head()

,state/region,ages,year,population,state,area (sq. mi)
28,AL,total,2000,4452173.0,Alabama,52423.0
68,AK,total,2000,627963.0,Alaska,656425.0
124,AZ,total,2000,5160586.0,Arizona,114006.0
162,AR,total,2000,2678588.0,Arkansas,53182.0
220,CA,total,2000,33987977.0,California,163707.0


## Re-index the data on state and find population density and display by density in ascending order

In [38]:
FY00.set_index('state', inplace=True)
density = FY00['population'] / FY00['area (sq. mi)']
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8412.441176
Puerto Rico             1084.098151
New Jersey               966.592639
Rhode Island             679.785113
Connecticut              615.399892
dtype: float64

The result is a ranking of US states plus Washington, DC, and Puerto Rico in order of their 2010 population density, in residents per square mile.
We can see that by far the densest region in this dataset is Washington, DC (i.e., the District of Columbia); among states, the densest is New Jersey.

We can also check the end of the list:

In [39]:
density.tail()

state
South Dakota    9.800755
North Dakota    9.080434
Montana         6.146192
Wyoming         5.053262
Alaska          0.956641
dtype: float64

We see that the least dense state, by far, is Alaska, averaging slightly over one resident per square mile.

This type of messy data merging is a common task when trying to answer questions using real-world data sources.
I hope that this example has given you an idea of the ways you can combine tools we've covered in order to gain insight from your data!